In [12]:
import cv2
import numpy as np

def text_to_bits(text):
    return ''.join(f'{ord(c):08b}' for c in text)

def bits_to_text(bits):
    chars = [bits[i:i+8] for i in range(0, len(bits), 8)]
    return ''.join(chr(int(b, 2)) for b in chars)

def encode_lsb(image, message, password):
    full_message = f"{password}:{message}"
    binary_msg = text_to_bits(full_message)
    msg_len = len(binary_msg)

    if msg_len > image.size:
        raise ValueError("Message too long to encode in image.")

    flat_img = image.flatten()
    for i in range(msg_len):
        flat_img[i] = np.uint8((int(flat_img[i]) & 0b11111110) | int(binary_msg[i]))
 # LSB replacement

    encoded_img = flat_img.reshape(image.shape)
    return encoded_img

def decode_lsb(image):
    flat_img = image.flatten()
    len_bits = ''.join(str(flat_img[i] & 1) for i in range(32))
    msg_len = int(len_bits, 2)
    print("🔎 Decoded message length (in bits):", msg_len)
    print("🖼️ Image capacity (in bits):", len(flat_img))
    if msg_len + 32 > len(flat_img):
        raise ValueError("Decoded message length is too large or image is corrupted.")

    # Now read only the number of bits that make up the message
    bits = ''.join(str(flat_img[i + 32] & 1) for i in range(msg_len))


    try:
        text = bits_to_text(bits)
        sep_index = text.find(':')
        if sep_index == -1:
            raise ValueError("No password/message separator found.")

        password, actual_message = text[:sep_index], text[sep_index + 1:]
        return password, actual_message
    except Exception as e:
        raise ValueError("Decryption failed: Possibly wrong image or corrupted data.")

# --- Main Program ---
img_path = input("Enter image path (e.g., 'mypic.png'): ")
img = cv2.imread(img_path)

if img is None:
    raise FileNotFoundError("Image not found.")

message = input("Enter secret message: ")
password = input("Set a password for message: ")

# Encode and save
encoded = encode_lsb(img.copy(), message, password)
cv2.imwrite("lsb_encrypted_image.png", encoded)
print("Message encoded and saved as 'lsb_encrypted_image.png'.")

# Decode
decode = input("Do you want to decode the image? (yes/no): ").strip().lower()
if decode == "yes":
    img2 = cv2.imread("lsb_encrypted_image.png")
    entered_password = input("Enter password to decrypt: ")

    try:
        saved_password, secret = decode_lsb(img2)
        if entered_password == saved_password:
            print("Decrypted message:", secret)
        else:
            print("Wrong password. Access denied.")
    except Exception as e:
        print("Error:", e)


Enter image path (e.g., 'mypic.png'): mypic.png
Enter secret message: hello from india
Set a password for message: 123
Message encoded and saved as 'lsb_encrypted_image.png'.
Do you want to decode the image? (yes/no): yes
Enter password to decrypt: 123
🔎 Decoded message length (in bits): 825373498
🖼️ Image capacity (in bits): 1769472
Error: Decoded message length is too large or image is corrupted.
